In [1]:
import sys
import os
# gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
# sys.path.append(gems_tco_path)

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle 

import GEMS_TCO
from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings
from GEMS_TCO import load_data_local_computer

import torch
from collections import defaultdict

import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import copy                    # clone tensor


In [2]:
lat_lon_resolution = [10,10]
day = 5
mm_cond_number = 20

years = ['2024']
month_range =[7,8]
idx_for_datamap= [ 8*(day-1),8*day]

instance = load_data_local_computer()
map, ord_mm, nns_map= instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)
analysis_data_map, aggregated_data = instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap=[0,8])


In [12]:
aggregated_data[ list([1,2,3]),:]

tensor([[  5.0250, 110.0250, 273.3987,  21.0000],
        [  5.0250, 119.5250, 262.7728,  21.0000],
        [  9.5250, 110.5250, 267.5342,  21.0000]], dtype=torch.float64)

### quick load from amarel

In [22]:
input_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/Exercises/st_model/estimates"

# input_filename = "vecc_extra_estimates_50_july24.pkl"
# input_filename = "vecc_inter_estimates_1250_july24.pkl"

input_filename = "vecc_inter_estimates_5000_july24.pkl"
# input_filename = "estimation_200_july24.pkl"
input_filename = "full_estimation_1250_july24.pkl"
input_filepath = os.path.join(input_path, input_filename)
# Load pickle
with open(input_filepath, 'rb') as pickle_file:
    amarel_map1250= pickle.load(pickle_file)

# Assuming df_1250 is your DataFrame
df_1250 = pd.DataFrame()
for key in amarel_map1250:
    tmp = pd.DataFrame(amarel_map1250[key][0].reshape(1, -1), columns=['sigmasq', 'range_lat', 'range_lon', 'advec_lat', 'advec_lon', 'beta', 'nugget'])
    tmp['loss'] = amarel_map1250[key][1]
    df_1250 = pd.concat((df_1250, tmp), axis=0)

# Generate date range
date_range = pd.date_range(start='07-01-24', end='07-31-24')

# Ensure the number of dates matches the number of rows in df_1250
if len(date_range) == len(df_1250):
    df_1250.index = date_range
else:
    print("The number of dates does not match the number of rows in the DataFrame.")

print(df_1250.head())
df = df_1250
# Save DataFrame to CSV
output_filename = 'vecchia_inter_estimates_1250_july24.csv'
output_csv_path = os.path.join(input_path, output_filename)
df.to_csv(output_csv_path, index=False)


              sigmasq  range_lat  range_lon  advec_lat  advec_lon      beta  \
2024-07-01  24.793444   1.584529   1.718248   0.009089  -0.107299  0.131038   
2024-07-02  24.424301   1.997055   1.942683   0.043588  -0.072679  0.137124   
2024-07-03  26.009497   1.215236   1.558868   0.023392  -0.150548  0.199850   
2024-07-04  24.701347   1.612308   1.822960  -0.164069  -0.237443  0.131595   
2024-07-05  22.598671   2.901185   3.722327  -0.011729  -0.152072  0.072866   

              nugget          loss  
2024-07-01  2.717239  14068.529297  
2024-07-02  1.513148  12357.715820  
2024-07-03  2.890678  14948.140625  
2024-07-04  3.636499  14786.204102  
2024-07-05  2.397249  12096.261719  


Optimization vecchia

In [19]:
instance.run_full(params, optimizer, scheduler, instance.matern_cov_anisotropy_v05, epochs=3000)

Epoch 1, Gradients: [-8.3266163e-01 -3.9050370e-01 -2.8237242e-01 -6.4676173e-04
 -3.2409048e-03  4.5490393e-01 -4.9839094e-01]
 Loss: 2548.8592623141867, Parameters: [ 2.51785984e+01  2.00787091e+00  2.10355711e+00  1.28148985e-03
 -7.98523873e-02  1.08808674e-01  3.60232282e+00]
Converged at epoch 1
Epoch 2, Gradients: [-8.3268803e-01 -3.9018026e-01 -2.8215161e-01 -6.4333249e-04
 -3.2253459e-03  4.5356691e-01 -4.9834314e-01]
 Loss: 2548.859166043867, full Parameters: [ 2.51787128e+01  2.00794053e+00  2.10362983e+00  1.28198182e-03
 -7.98518509e-02  1.08804606e-01  3.60241866e+00]
FINAL STATE: Epoch 2, Gradients: [-8.3268803e-01 -3.9018026e-01 -2.8215161e-01 -6.4333249e-04
 -3.2253459e-03  4.5356691e-01 -4.9834314e-01]
 Loss: 2548.859166043867, full Parameters: [ 2.51787128e+01  2.00794053e+00  2.10362983e+00  1.28198182e-03
 -7.98518509e-02  1.08804606e-01  3.60241866e+00]


[array([ 2.51787128e+01,  2.00794053e+00,  2.10362983e+00,  1.28198182e-03,
        -7.98518509e-02,  1.08804606e-01,  3.60241866e+00], dtype=float32),
 2548.859166043867]

# optimize

```    srun --cpus-per-task=3 --mem=5G --time=05:00:00 python /home/jl2815/tco/exercise_25/st_model/fit_st_vecc_v05_int414.py --v 0.5 --lr 0.01 --epochs 3000 --space "20, 20" --days 1 --mm-cond-number 10 --nheads 200 --params "24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34"    ```


space 10 10   lr 0.03 for vecc and step size 80   then sec 21  15

space 8 8    maybe lr 0.02    then sec 120 40
             if lr 0.005   sec 250 (accuracy is worse)



In [ ]:
/opt/anaconda3/envs/faiss_env/bin/python /Users/joonwonlee/Documents/GEMS_TCO-1/Exercises/st_model/vecc_opt.py --space "20,20" --v 0.5 --lr 0.01 --epochs 2000 --days 1 --mm-cond-number 10 --nheads 100

/opt/anaconda3/envs/faiss_env/bin/python /Users/joonwonlee/Documents/GEMS_TCO-1/Exercises/st_model/vecc_opt.py --space "10,10" --v 0.5 --lr 0.01 --epochs 3000 --days 1 --mm-cond-number 10 --nheads 100


/opt/anaconda3/envs/faiss_env/bin/python /Users/joonwonlee/Documents/GEMS_TCO-1/Exercises/st_model/vecc_opt.py --space "8,8" --v 0.5 --lr 0.01 --epochs 3000 --days 1 --mm-cond-number 10 --nheads 100


In [ ]:
params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
params = torch.tensor(params, requires_grad=True)
nheads= 200
instance = kernels.model_fitting(
    smooth=0.5,
    input_map=analysis_data_map,
    aggregated_data=aggregated_data,
    nns_map=nns_map,
    mm_cond_number=mm_cond_number,
    nheads= nheads
)

# optimizer = optim.Adam([params], lr=0.01)  # For Adam
optimizer, scheduler = instance.optimizer_fun( params, lr=0.01, betas=(0.9, 0.99), eps=1e-8, step_size=10, gamma=0.9)  
instance.run_vecc_interpolate(params, optimizer, scheduler, instance.matern_cov_anisotropy_v05, epochs=3000)




Epoch 1, Gradients: [  0.5812249  19.981815   11.620518    2.1510413  18.509764  447.8913
   5.8959665]
 Loss: 2588.8354945381675, Parameters: [ 2.442e+01  1.920e+00  1.920e+00  1.000e-03 -4.500e-02  2.370e-01
  3.340e+00]
Epoch 101, Gradients: [-0.8209724  -1.028479   -0.7551035  -0.04002035 -0.02916028  6.1020327
 -1.0398538 ]
 Loss: 2549.4410424626913, Parameters: [ 2.4837215e+01  1.8965260e+00  1.9835217e+00  3.4586992e-04
 -8.0771782e-02  1.1651777e-01  3.4273267e+00]
Epoch 201, Gradients: [-0.8235874  -0.691919   -0.4959467  -0.0016299  -0.00983667  1.2110153
 -0.6988743 ]
 Loss: 2549.038469945362, Parameters: [ 2.50627880e+01  1.96428633e+00  2.05704403e+00  9.47781198e-04
 -8.02353993e-02  1.11530036e-01  3.53635502e+00]
Epoch 301, Gradients: [-0.8261704  -0.5165033  -0.3714979  -0.0011931  -0.00638401  0.809959
 -0.5589241 ]
 Loss: 2548.9165855832653, Parameters: [ 2.5141787e+01  1.9913456e+00  2.0860329e+00  1.1571334e-03
 -7.9993919e-02  1.0981011e-01  3.5778422e+00]
Epoch 4

TypeError: model_fitting.run_full() missing 1 required positional argument: 'covariance_function'

# Vecchia experiments here

beat the b2

In [ ]:
/opt/anaconda3/envs/faiss_env/bin/python /Users/joonwonlee/Documents/GEMS_TCO-1/Exercises/vecc_beat2.py

/opt/anaconda3/envs/faiss_env/bin/python /Users/joonwonlee/Documents/GEMS_TCO-1/Exercises/vecc_beat3.py

In [5]:
lat_lon_resolution = [4,4]
years = ['2024']
month_range =[7,8]
nheads = 200

result_2 = [0]*3
result_1 = [0]*3
for day in range(1,2):
    print(f'\n Day {day} data size per day: { (200/lat_lon_resolution[0])*(100/lat_lon_resolution[0])  } \n')

    # parameters
    mm_cond_number = 10+day
    idx_for_datamap= [ 8*(day-1),8*day]
    # params = [ 27.25, 2.18, 2.294, 4.099e-4, -0.07915, 0.0999, 3.65]   #200
    params = list(df.iloc[day-1][:-1])
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)

    # data
    instance = load_data_local_computer()
    map, ord_mm, nns_map= instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)
    analysis_data_map, aggregated_data = instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap= idx_for_datamap)

    # different approximations
    key_order = [0,1,2,4,3,5,7,6]
    keys = list(analysis_data_map.keys())
    reordered_dict = {keys[key]: analysis_data_map[keys[key]] for key in key_order}

    instance_ori = kernels.vecchia_experiment(0.5, analysis_data_map, aggregated_data,nns_map,mm_cond_number, nheads)
    out = instance_ori.full_likelihood(params, aggregated_data[:,:4],aggregated_data[:,2], instance_ori.matern_cov_anisotropy_v05)
    print(f'full: {out}')  

    out1 = instance_ori.vecchia_interpolation_1to6(params,  instance_ori.matern_cov_anisotropy_v05)
    print(f'vecc t-1, t+1: {out1}')  

    #out22 = instance_ori.vecchia_b2(params,  instance_ori.matern_cov_anisotropy_v05)
    #print(f'vecc two lags: {out22}')  

    instance = kernels.vecchia_experiment(0.5, reordered_dict, aggregated_data,nns_map,mm_cond_number, nheads)

    out2 = instance.vecchia_b2(params,  instance.matern_cov_anisotropy_v05)
    print(f'vecc two lags: {out2}')  

    out3 = instance.vecchia_competitor(params,  instance.matern_cov_anisotropy_v05)
    print(f'vecc competitor: {out3}')  


    approx_map = {0: 'vecc t-1, t+1', 1: 'two lag', 2:'competitor'}

    tmp_result = [ torch.abs(out-out1), torch.abs(out-out2) , torch.abs(out-out3)]
    stacked_tensor = torch.stack(tmp_result)
    top2_indices = torch.topk(stacked_tensor, 2, largest=False).indices

    best_index = top2_indices[0].item()
    second_best_index = top2_indices[1].item()

    # Update the result for the best approximation
    result_1[best_index] += 1
    result_2[second_best_index] +=1
    # Print the results
    print(f'\n\n Day {day} full likelihood: {out}\n parameters: {params.tolist()} \n')
    print(f'Best approximation: {approx_map[best_index]} with abs_diff: {stacked_tensor[best_index]}')
    print(f'Second best approximation: {approx_map[second_best_index]} with abs_diff: {stacked_tensor[second_best_index]}')



 Day 1 data size per day: 1250.0 

full: 14068.486752749093
vecc t-1, t+1: 14026.824791852318
vecc two lags: 14179.493975372936
vecc two lags: 14152.955435277818
vecc competitor: 14155.957972675671


 Day 1 full likelihood: 14068.486752749093
 parameters: [24.79344367980957, 1.584528923034668, 1.7182477712631226, 0.009088504128158092, -0.10729944705963135, 0.131037637591362, 2.717238664627075] 

Best approximation: vecc t-1, t+1 with abs_diff: 41.66196089677578
Second best approximation: two lag with abs_diff: 84.46868252872446


Invesitagate how many lags should I use

In [ ]:
lat_lon_resolution = [4,4]
years = ['2024']
month_range =[7,8]
nheads = 200

result_2 = [0]*5
result_1 = [0]*5
for day in range(1,2):
    print(f'\n day {day} data size per day: { (200/lat_lon_resolution[0])*(100/lat_lon_resolution[0])  } \n')

    # parameters
    mm_cond_number = 10+day
    idx_for_datamap= [ 8*(day-1),8*day]
    # params = [ 27.25, 2.18, 2.294, 4.099e-4, -0.07915, 0.0999, 3.65]   #200
    params = list(df.iloc[day-1][:-1])
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)

    # data
    instance = load_data_local_computer()
    map, ord_mm, nns_map= instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)
    analysis_data_map, aggregated_data = instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap= idx_for_datamap)

    # different approximations
    key_order = [0,1,2,4,3,5,7,6]
    keys = list(analysis_data_map.keys())
    reordered_dict = {keys[key]: analysis_data_map[keys[key]] for key in key_order}

    instance_ori = kernels.vecchia_experiment(0.5, analysis_data_map, aggregated_data,nns_map,mm_cond_number, nheads)
    out = instance_ori.full_likelihood(params, aggregated_data[:,:4],aggregated_data[:,2], instance_ori.matern_cov_anisotropy_v05)
    print(f'full: {out}')  # 15105

    out1 = instance_ori.vecchia_interpolation_1to6(params,  instance_ori.matern_cov_anisotropy_v05)
    print(f'vecc t-1, t+1: {out1}')  # 15105

    instance = kernels.vecchia_experiment(0.5, reordered_dict, aggregated_data,nns_map,mm_cond_number, nheads)

    out2 = instance.vecchia_b1(params,  instance.matern_cov_anisotropy_v05)
    print(f'vecc one lag : {out2}')  # 15105

    out3 = instance.vecchia_b2(params,  instance.matern_cov_anisotropy_v05)
    print(f'vecc two lags: {out3}')  # 15105

    out4 = instance.vecchia_b3(params,  instance.matern_cov_anisotropy_v05)
    print(f'vecc three lags: {out4}')  # 15105

    out5 = instance.vecchia_b4(params,  instance.matern_cov_anisotropy_v05)
    print(f'vecc four lags: {out5}')  # 15105

    approx_map = {0: 'vecc t-1, t+1', 1: 'one lag', 2:'two lags', 3:'three lags', 4:'four lags'}


    tmp_result = [torch.abs(out-out1), torch.abs(out-out2), torch.abs(out-out3), torch.abs(out-out4), torch.abs(out-out5)]
    stacked_tensor = torch.stack(tmp_result)
    top2_indices = torch.topk(stacked_tensor, 2, largest=False).indices

    best_index = top2_indices[0].item()
    second_best_index = top2_indices[1].item()

    # Update the result for the best approximation
    result_1[best_index] += 1
    result_2[second_best_index] +=1
    # Print the results
    print(f'\n\n Day {day} full likelihood: {out}\n parameters: {params.tolist()} \n')
    print(f'Best approximation: {approx_map[best_index]} with abs_diff: {stacked_tensor[best_index]}')
    print(f'Second best approximation: {approx_map[second_best_index]} with abs_diff: {stacked_tensor[second_best_index]}')




 day 1 data size per day: 1250.0 

full: 14068.486752749093
vecc t-1, t+1: 14026.824791852318
vecc one lag : 14170.382029741002
vecc two lags: 14152.955435277818
vecc three lags: 14175.718607333947
vecc four lags: 14176.37903307759


 Day 1 full likelihood: 14068.486752749093
 parameters: [24.79344367980957, 1.584528923034668, 1.7182477712631226, 0.009088504128158092, -0.10729944705963135, 0.131037637591362, 2.717238664627075] 


Best approximation: vecc t-1, t+1 with absolute difference 41.66196089677578
Second best approximation: two lags with absolute difference 84.46868252872446


In [23]:

approx_map[  torch.argmin( torch.stack(tmp_result)).item() ]

'vecc t-1, t+1'

In [ ]:
lat_lon_resolution = [10,10]

head100map = defaultdict(list)
nheads= 10
mm_cond_number = 10

result_2 = [0]*7
result_1 = [0]*7
for day in range(1,32):
    

    years = ['2024']
    month_range =[7,8]
    idx_for_datamap= [ 8*(day-1),8*day]

    instance = load_data_local_computer()
    map, ord_mm, nns_map= instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)
    analysis_data_map, aggregated_data = instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap= idx_for_datamap)

    params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
    params = torch.tensor(params, requires_grad=True)
    instance = kernels.vecchia_experiment(0.5, analysis_data_map, aggregated_data, nns_map, mm_cond_number, nheads)


    out = instance.full_likelihood(params, aggregated_data[:,:4],aggregated_data[:,2], instance.matern_cov_anisotropy_v05)
    print(out)  # 15105

    out1 = instance.vecchia_interpolation_1to6(params,  instance.matern_cov_anisotropy_v05)
    print(f'vecc t-1, t+1: {out1}')  # 15105

    out2 = instance.vecchia_b1(params,  instance.matern_cov_anisotropy_v05)
    print(f'vecc one lag : {out2}')  

    out3 = instance.vecchia_b2(params,  instance.matern_cov_anisotropy_v05)
    print(f'vecc two lags: {out3}')  

    out4 = instance.vecchia_b3(params,  instance.matern_cov_anisotropy_v05)
    print(f'vecc three lags: {out4}')  

    out5 = instance.vecchia_b4(params,  instance.matern_cov_anisotropy_v05)
    print(f'vecc four lags: {out5}')  

    out6 = instance.vecchia_b5(params, instance.matern_cov_anisotropy_v05)
    print(f'vecc five lags: {out6}') 

    out7 = instance.vecchia_b6(params, instance.matern_cov_anisotropy_v05)
    print(f'vecc six lags: {out7}')  

    approx_map = {0: 'vecc t-1, t+1', 1: 'one lag', 2:'two lags', 3:'three lags', 4:'four lags', 5:'five lags' ,6:'six lags' }



    tmp_result = [torch.abs(out-out1), torch.abs(out-out2), torch.abs(out-out3), torch.abs(out-out4), torch.abs(out-out5),  torch.abs(out-out6) , torch.abs(out-out7)]
    stacked_tensor = torch.stack(tmp_result)
    top2_indices = torch.topk(stacked_tensor, 2, largest=False).indices

    best_index = top2_indices[0].item()
    second_best_index = top2_indices[1].item()

    # Update the result for the best approximation
    result_1[best_index] += 1
    result_2[second_best_index] +=1
    # Print the results
    print(f'\n\n Day {day} full likelihood: {out}\n parameters: {params.tolist()} \n')
    print(f'Best approximation: {approx_map[best_index]} with abs_diff: {stacked_tensor[best_index]}')
    print(f'Second best approximation: {approx_map[second_best_index]} with abs_diff: {stacked_tensor[second_best_index]}')



tensor(2588.8355, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor(2598.6200, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2596.4554, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2597.2973, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2597.4288, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2597.6245, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2597.7251, dtype=torch.float64, grad_fn=<AddBackward0>)
 day 2 full 2588.835494538167 best 2-7.619876935808406
tensor(2325.3057, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor(2338.3503, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2337.2224, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2337.3480, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2337.8504, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2337.7865, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2337.9979, dtype=torch.float64, grad_fn=<AddBackward0>)
 day 3 full 2325.3057231697794 best 2-11.916687282841394
tensor

In [ ]:
bmap = {}  ## key (mm_cond_number, headn) values: b1 2 b3 b4 b5 b6 
bmap[(10,10)] = [3,2,11,4,1,4,0]
bmap[(10,20)] = [9,4,8,3,1,2,0]
bmap[(10,30)] = [10,3,4,5,1,3,0]
bmap[(10,40)] = [12,4,4,4,2,3,0]
bmap[(10,50)] = [13,3,4,4,2,2,0]  # [0, 5, 7, 6, 2, 5, 0]

{(10, 10): [3, 2, 11, 4, 1, 4, 0],
 (10, 20): [9, 4, 8, 3, 1, 2, 0],
 (10, 30): [10, 3, 4, 5, 1, 3, 0]}

Fit the model

In [95]:
params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
params = torch.tensor(params, requires_grad=True)

instance = torch_vecchia_exp(analysis_data_map, params, nns_map, mm_cond_number)

# optimizer = optim.Adam([params], lr=0.01)  # For Adam
optimizer = optim.Adam([params],lr=0.01, betas=(0.9, 0.8), eps=1e-8)
scheduler = StepLR(optimizer, step_size=20, gamma=0.9)

# Example function to compute out1
def compute_out1(params):
    # Compute the output using your function
    # nll = instance.full_likelihood(params, instance.matern_advec_beta_cov )
    nll = instance.vecchia_local4(params, instance.matern_cov_ani, 200 )
    return nll

# Training loop
prev_loss = float('inf')
tol = 1e-4  # Convergence tolerance
for epoch in range(600):  # Number of epochs
    optimizer.zero_grad()  # Zero the gradients 
    
    loss = compute_out1(params)
    loss.backward()  # Backpropagate the loss
    
    # Print gradients and parameters every 10th epoch
    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
    
    # print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
    
    optimizer.step()  # Update the parameters
    scheduler.step()
    # Check for convergence
    if abs(prev_loss - loss.item()) < tol:
        print(f"Converged at epoch {epoch}")
        print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
    
        break
    
    prev_loss = loss.item()

print('Training complete.') 

Epoch 1, Gradients: [  -9.955391  207.88681    -2.595461 -169.72203   273.39944   771.1119
  -79.33867 ]
 Loss: 15135.7587890625, Parameters: [ 2.442e+01  1.920e+00  1.920e+00  1.000e-03 -4.500e-02  2.370e-01
  3.340e+00]
Epoch 101, Gradients: [ -1.4867018    0.10161591   1.1231489    3.3106918  -11.375813
  -7.5312614    7.9387875 ]
 Loss: 15014.2587890625, Parameters: [ 2.5338207e+01  1.3597493e+00  1.7280715e+00  1.9441580e-02
 -1.6756412e-01  1.8955134e-01  3.3110123e+00]
Converged at epoch 189
Epoch 190, Gradients: [-1.2997863  -0.27633095 -0.09813547  0.49712753 -0.5989456  -1.545433
 -0.09783697]
 Loss: 15012.7626953125, Parameters: [ 2.5795250e+01  1.3089702e+00  1.6588148e+00  1.8806925e-02
 -1.6661680e-01  1.9616681e-01  3.1115689e+00]
Training complete.


In [ ]:
params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
params = torch.tensor(params, requires_grad=True)

instance = torch_vecchia_exp(analysis_data_map, params, nns_map, mm_cond_number)

# optimizer = optim.Adam([params], lr=0.01)  # For Adam
optimizer = optim.Adam([params],lr=0.01, betas=(0.9, 0.8), eps=1e-8)
scheduler = StepLR(optimizer, step_size=20, gamma=0.9)

# Example function to compute out1
def compute_out1(params):
    # Compute the output using your function
    # nll = instance.full_likelihood(params, instance.matern_advec_beta_cov )
    nll = instance.vecchia_local3(params, instance.matern_cov_ani )
    return nll

# Training loop
prev_loss = float('inf')
tol = 1e-4  # Convergence tolerance
for epoch in range(500):  # Number of epochs
    optimizer.zero_grad()  # Zero the gradients 
    
    loss = compute_out1(params)
    loss.backward()  # Backpropagate the loss
    
    # Print gradients and parameters every 10th epoch
    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
    
    # print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
    
    optimizer.step()  # Update the parameters
    scheduler.step()
    # Check for convergence
    if abs(prev_loss - loss.item()) < tol:
        print(f"Converged at epoch {epoch}")
        print(f'Epoch {epoch+1}, Gradients: {params.grad.numpy()}\n Loss: {loss.item()}, Parameters: {params.detach().numpy()}')
        break
    
    prev_loss = loss.item()

print('Training complete.') 

 
# vecchia local 2  332   epo 33.4s   25.55 2.61 2.68 0.16 0.03 2.7
#    24.89 2.06 2.24 1.36e-2 -5.63 e-2 0.10113 3.75
# vecchia cholesky local 380 epo 43.2  25 2.61 2.68 0.16 0.03 2.7

Epoch 1, Gradients: [-10.6224985 200.28781    -6.4957714 -78.33738   149.36243   605.0917
 -81.693665 ]
 Loss: 15183.4765625, Parameters: [ 2.442e+01  1.920e+00  1.920e+00  1.000e-03 -4.500e-02  2.370e-01
  3.340e+00]
Epoch 101, Gradients: [-1.9947629  4.9151707  6.4719105 -1.286026  -2.0892706  2.9007835
  1.0226293]
 Loss: 15077.75, Parameters: [25.32368     1.3661038   1.7502507   0.04647188 -0.12653434  0.17788467
  3.327231  ]
Converged at epoch 135
Training complete.


Experiment results

## 200 x 8

lr 0.001 without scheduler  same as lr, step_size, gamma  0.01 40 0.5  (9.8s)

 Loss: 2549.066650390625, full Parameters: [ 2.48777485e+01  2.05998826e+00  2.16013098e+00  2.20775465e-03
 -7.89414570e-02  1.05411254e-01  3.75236106e+00]

 lr 0.01  step size 40  betas 0.9 , 0.8 gamma 0.9  30 s

  Loss: 2547.1728515625, full Parameters: [ 2.7377291e+01  2.2077193e+00  2.3204505e+00  1.0307773e-03
 -8.0311157e-02  9.8579854e-02  3.6677265e+00]

 lr 0.01  step size 10 betas 0.9 , 0.8 gamma 0.9  30 s
  Loss: 2548.87841796875, full Parameters: [ 2.5092268e+01  2.0689390e+00  2.1694989e+00  2.0285936e-03
 -7.9028614e-02  1.0501490e-01  3.7373385e+00]
Training full likelihood complete.   11.8 sc

 lr 0.01  step size 20 betas 0.9 , 0.8 gamma 0.9  30 s
 Loss: 2548.15283203125, full Parameters: [ 2.59814014e+01  2.12175608e+00  2.22699022e+00  1.73025124e-03
 -7.93599486e-02  1.02427535e-01  3.70715070e+00]



lr 0.01  step size 20 beta 0.9 0.99 gamma 0.9
 Loss: 2548.18603515625, full Parameters: [ 2.5938652e+01  2.1110108e+00  2.2155209e+00  1.5893303e-03
 -7.9482891e-02  1.0297947e-01  3.6958976e+00]
 21.6

lr 0.01  step size 20 beta 0.9 0.8 gamma 0.9
 Loss: 2548.15283203125, full Parameters: [ 2.59814014e+01  2.12175608e+00  2.22699022e+00  1.73025124e-03
 -7.93599486e-02  1.02427535e-01  3.70715070e+00]
 22.9 s

lr 0.01  step size 10 beta 0.9 0.99 gamma 0.9
Loss: 2548.95361328125, full Parameters: [ 2.5118145e+01  1.9827319e+00  2.0768294e+00  1.0898338e-03
 -8.0070712e-02  1.1034889e-01  3.5647078e+00]


## 1250 x 8

1250* 8 55m using constant learning rate 0.0001 
Loss: 14068.798828125, full Parameters: [ 2.46198387e+01  1.61719894e+00  1.76454413e+00  8.55297223e-03
 -1.08275235e-01  1.28809512e-01  2.80795789e+00]

1250* 8 10m 32s
lr 0.01  step size 40 beta 0.9 0.8 gamma 0.9
  Loss: 14068.1953125, full Parameters: [ 2.5030930e+01  1.6107724e+00  1.7573007e+00  8.8407323e-03
 -1.0820019e-01  1.2936097e-01  2.7430327e+00]
Training full likelihood complete.

9m 33s
lr 0.01  step size 20 beta 0.9 0.8 gamma 0.9
 Loss: 14068.29296875, full Parameters: 
 [ 2.4933689e+01  1.6009743e+00  1.7502663e+00  9.2404895e-03 -1.0737537e-01  1.2953614e-01 
  2.7420275e+00]
Training full likelihood complete.

#### high resolution data might benefits from larger step size high resolution data often provides 
#### more stable gradients, so larger step size less likely to cause significant fluctuations
14n 41.8s
lr 0.01  step size 10 beta 0.9 0.99 gamma 0.9

FINAL STATE: Epoch 199, 
 Loss: 14068.8828125, full Parameters: 
 [ 2.4707581e+01  1.6489888e+00  1.7993137e+00  8.4043797e-03 -1.0836436e-01  1.2655504e-01  
 2.8416286e+00]

#### beta 0.9 0.99 might be too conservative for high resolution data
13m 44.8s
lr 0.01  step size 20 beta 0.9 0.99 gamma 0.9

 Loss: 14068.318359375, full Parameters: [ 2.4938175e+01  1.6203119e+00  1.7678342e+00  8.6686825e-03
 -1.0813228e-01  1.2845081e-01  2.7731323e+00]


18m
lr 0.01  step size 40 beta 0.9 0.99 gamma 0.9

 Loss: 14067.970703125, full Parameters: [ 2.5205673e+01  1.6159834e+00  1.7630767e+00  8.7957922e-03
 -1.0802399e-01  1.2862283e-01  2.7390635e+00]

9m 52s
lr 0.01  step size 20 beta 0.9 0.8 gamma 0.9

Loss: 14068.29296875, full Parameters: [ 2.4933689e+01  1.6009743e+00  1.7502663e+00  9.2404895e-03
 -1.0737537e-01  1.2953614e-01  2.7420275e+00]
Training full likelihood complete.